In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
import os
import random
from copy import deepcopy
import _pickle as pickle
import gc
from multiprocess import Pool
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.preprocessing import KBinsDiscretizer

from tensorflow.keras.optimizers import Adam, SGD
def save(file,name, folder = ""):
    if folder != "":
        outfile = open('./'+folder+'/'+name+'.pickle', 'wb')
    else:
        outfile = open(name+'.pickle', 'wb')
    pickle.dump(file, outfile, protocol=4)
    outfile.close
    
def load(name, folder = ""):
    
    if folder != "":
        outfile = open('./'+folder+'/'+name+'.pickle', 'rb')
    else:
        outfile = open(name+'.pickle', 'rb')
    file = pickle.load(outfile)
    outfile.close
    return file

class Discretiser:
    def __init__(self, nbins):
        self.nbins = nbins-1
        self.map_to = np.arange(self.nbins)/self.nbins
        
    def fit(self, X):
        ## X is a one dimension np array
        self.map_from = np.quantile(X, self.map_to)
        
    def transform(self, X):
        X1 = (np.interp(X, self.map_from, self.map_to, left=0, right=1, period=None) * self.nbins).astype(int)
        return X1
    
from tf_transformers2 import *
from tensorflow.keras.layers import Input, Dense, Dropout, TimeDistributed, LSTM
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

In [ ]:
## features present
exercise to predict    ## cat
part of exercise       ## cat
gtag of exercise       ## cat
cluster of exercise    ## cat
qmean on user          ## num
qmean on content
container              ## num
timestamp              ## num

## content / user

## feature past
avg correctness
number explanation
number question
number lecture
avg elapsed time

## per part
avg correct
number explained
number lecture
number question
avg elapsed time
time since first question
time since last question
time since last lecture

## Per cluster
avg correct
number explained
number lecture
number question
avg elapsed time
time since first question
time since last question
time since last lecture

In [ ]:
## Segments          28
user                 1
content              1
parts                6
cluster              20

## Time windows      17 possibles
history
# based on id
first five / ten /twenty / 100
last five / ten / twenty / 100
# Based on time
last 1/5/12/24/168/720 hours
first 1/5/12/24/168/720 hours

## Kpis              4 possibles 
interaction_type
time spent
time elapsed
explanation

## Kpis type         7 possibles
count
mean
hmean
std
min
max
slope - % improvement



#####################################
## Total 13328 features

In [ ]:
def get_user_features(user_dic):
    import numpy as np
    import matplotlib.pyplot as plt
    import pandas as pd
    from tqdm.auto import tqdm
    from features_util import prepare, get_current_past, get_features
    import random
    user = prepare(user_dic)
    
    ##sampling
    seq_len = len(user['exercise_id'])
    r = random.randint(min(3, seq_len), min(10, seq_len))
    ids_to_fetch = np.random.choice(list(range(seq_len)), size = r, replace = False) 
               
    mat = np.zeros((r, 1049))
    for j, i in enumerate(tqdm(ids_to_fetch)):
        current, past = get_current_past(user, i)
        final = get_features(current, past)
        k = list(final.keys())
        v = list(final.values())
        mat[j] = v
    df = pd.DataFrame(mat, columns= k)
    return df

In [ ]:
from multiprocess import Pool
import pandas as pd
from tqdm.notebook import tqdm
from features_util import load, save
import os

p = Pool(10)
for elt in tqdm(os.listdir('user_batch_2000')):
    if not(elt in os.listdir('user_batch_lgb_2000')):
        dico = load(elt.split('.')[0], 'user_batch_2000')
        list_dico = [dico[elt] for elt in dico]
        df = p.map(get_user_features, list_dico)
        df = pd.concat(df)
        save(df, elt.split('.')[0], 'user_batch_lgb_2000')
p.close()

In [2]:
df_train = []
for i in tqdm(range(1,30)):
    df_train.append(load('batch_'+str(i), 'user_batch_lgb_2000'))
df_train = pd.concat(df_train)  
df_test = []
for i in tqdm(range(40,50)):
    df_test.append(load('batch_'+str(i), 'user_batch_lgb_2000'))
df_test = pd.concat(df_test)

In [3]:
df_train = df_train[df_train['correctness']!=-1]
df_test = df_test[df_test['correctness']!=-1]
X_train, y_train = df_train[[elt for elt in df_train.columns if not(elt == 'correctness')]], df_train['correctness'].values
X_test, y_test = df_test[[elt for elt in df_test.columns if not(elt == 'correctness')]], df_test['correctness'].values

del df_train
del df_test
gc.collect()

38

In [4]:
import lightgbm as lgb
clf = lgb.LGBMClassifier(max_depth = -1, n_estimators = 10000, n_jobs = 12, silent = False)
clf.fit(X_train, y_train, eval_set =(X_test, y_test), eval_metric = 'auc')

[LightGBM] [Info] Number of positive: 201628, number of negative: 167369
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 2.884346 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 207203
[LightGBM] [Info] Number of data points in the train set: 368997, number of used features: 995
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.546422 -> initscore=0.186223
[LightGBM] [Info] Start training from score 0.186223
[1]	valid_0's auc: 0.739731	valid_0's binary_logloss: 0.67236
[2]	valid_0's auc: 0.744644	valid_0's binary_logloss: 0.658666
[3]	valid_0's auc: 0.745662	valid_0's binary_logloss: 0.647414
[4]	valid_0's auc: 0.74731	valid_0's binary_logloss: 0.637959
[5]	valid_0's auc: 0.747904	valid_0's binary_logloss: 0.63007
[6]	valid_0's auc: 0.748403	valid_0's binary_logloss: 0.623437
[7]	valid_0's auc: 0.748834	valid_0's binary_logloss: 0.617879
[8]	valid_0's auc: 0.749293	valid_0's binary_logloss: 0.61315

[123]	valid_0's auc: 0.758177	valid_0's binary_logloss: 0.580493
[124]	valid_0's auc: 0.758178	valid_0's binary_logloss: 0.580491
[125]	valid_0's auc: 0.758192	valid_0's binary_logloss: 0.580476
[126]	valid_0's auc: 0.758196	valid_0's binary_logloss: 0.580473
[127]	valid_0's auc: 0.758208	valid_0's binary_logloss: 0.580464
[128]	valid_0's auc: 0.758204	valid_0's binary_logloss: 0.580465
[129]	valid_0's auc: 0.758209	valid_0's binary_logloss: 0.580459
[130]	valid_0's auc: 0.758257	valid_0's binary_logloss: 0.580408
[131]	valid_0's auc: 0.758251	valid_0's binary_logloss: 0.580413
[132]	valid_0's auc: 0.758276	valid_0's binary_logloss: 0.580383
[133]	valid_0's auc: 0.758281	valid_0's binary_logloss: 0.580376
[134]	valid_0's auc: 0.758286	valid_0's binary_logloss: 0.580371
[135]	valid_0's auc: 0.758274	valid_0's binary_logloss: 0.580383
[136]	valid_0's auc: 0.758252	valid_0's binary_logloss: 0.5804
[137]	valid_0's auc: 0.758231	valid_0's binary_logloss: 0.580417
[138]	valid_0's auc: 0.7582

[250]	valid_0's auc: 0.758133	valid_0's binary_logloss: 0.580505
[251]	valid_0's auc: 0.758142	valid_0's binary_logloss: 0.580495
[252]	valid_0's auc: 0.758138	valid_0's binary_logloss: 0.580497
[253]	valid_0's auc: 0.758126	valid_0's binary_logloss: 0.580513
[254]	valid_0's auc: 0.758131	valid_0's binary_logloss: 0.580512
[255]	valid_0's auc: 0.758133	valid_0's binary_logloss: 0.580509
[256]	valid_0's auc: 0.758146	valid_0's binary_logloss: 0.580498
[257]	valid_0's auc: 0.758157	valid_0's binary_logloss: 0.580491
[258]	valid_0's auc: 0.758184	valid_0's binary_logloss: 0.580461
[259]	valid_0's auc: 0.758199	valid_0's binary_logloss: 0.580442
[260]	valid_0's auc: 0.758205	valid_0's binary_logloss: 0.580434
[261]	valid_0's auc: 0.758197	valid_0's binary_logloss: 0.580441
[262]	valid_0's auc: 0.758189	valid_0's binary_logloss: 0.580448
[263]	valid_0's auc: 0.75818	valid_0's binary_logloss: 0.580461
[264]	valid_0's auc: 0.758195	valid_0's binary_logloss: 0.580448
[265]	valid_0's auc: 0.758

[376]	valid_0's auc: 0.758104	valid_0's binary_logloss: 0.580555
[377]	valid_0's auc: 0.758097	valid_0's binary_logloss: 0.580559
[378]	valid_0's auc: 0.758094	valid_0's binary_logloss: 0.580561
[379]	valid_0's auc: 0.758076	valid_0's binary_logloss: 0.580579
[380]	valid_0's auc: 0.758066	valid_0's binary_logloss: 0.58059
[381]	valid_0's auc: 0.75808	valid_0's binary_logloss: 0.580578
[382]	valid_0's auc: 0.758085	valid_0's binary_logloss: 0.580575
[383]	valid_0's auc: 0.758065	valid_0's binary_logloss: 0.580593
[384]	valid_0's auc: 0.758079	valid_0's binary_logloss: 0.580586
[385]	valid_0's auc: 0.758062	valid_0's binary_logloss: 0.580597
[386]	valid_0's auc: 0.758062	valid_0's binary_logloss: 0.580597
[387]	valid_0's auc: 0.75805	valid_0's binary_logloss: 0.580607
[388]	valid_0's auc: 0.758043	valid_0's binary_logloss: 0.580614
[389]	valid_0's auc: 0.758027	valid_0's binary_logloss: 0.580627
[390]	valid_0's auc: 0.758018	valid_0's binary_logloss: 0.58063
[391]	valid_0's auc: 0.758025

[504]	valid_0's auc: 0.757578	valid_0's binary_logloss: 0.581017
[505]	valid_0's auc: 0.757579	valid_0's binary_logloss: 0.58102
[506]	valid_0's auc: 0.757578	valid_0's binary_logloss: 0.581025
[507]	valid_0's auc: 0.757576	valid_0's binary_logloss: 0.581029
[508]	valid_0's auc: 0.757571	valid_0's binary_logloss: 0.581033
[509]	valid_0's auc: 0.757566	valid_0's binary_logloss: 0.58104
[510]	valid_0's auc: 0.757555	valid_0's binary_logloss: 0.581047
[511]	valid_0's auc: 0.757552	valid_0's binary_logloss: 0.581055
[512]	valid_0's auc: 0.757555	valid_0's binary_logloss: 0.581049
[513]	valid_0's auc: 0.757551	valid_0's binary_logloss: 0.581053
[514]	valid_0's auc: 0.757552	valid_0's binary_logloss: 0.581054
[515]	valid_0's auc: 0.757556	valid_0's binary_logloss: 0.58105
[516]	valid_0's auc: 0.757551	valid_0's binary_logloss: 0.581052
[517]	valid_0's auc: 0.757534	valid_0's binary_logloss: 0.581069
[518]	valid_0's auc: 0.757542	valid_0's binary_logloss: 0.581063
[519]	valid_0's auc: 0.75753

...

[2891]	valid_0's auc: 0.751173	valid_0's binary_logloss: 0.590556
[2892]	valid_0's auc: 0.751174	valid_0's binary_logloss: 0.590561
[2893]	valid_0's auc: 0.751183	valid_0's binary_logloss: 0.590549
[2894]	valid_0's auc: 0.751179	valid_0's binary_logloss: 0.590556
[2895]	valid_0's auc: 0.751181	valid_0's binary_logloss: 0.590555
[2896]	valid_0's auc: 0.751186	valid_0's binary_logloss: 0.590552
[2897]	valid_0's auc: 0.751185	valid_0's binary_logloss: 0.590559
[2898]	valid_0's auc: 0.751179	valid_0's binary_logloss: 0.590565
[2899]	valid_0's auc: 0.751185	valid_0's binary_logloss: 0.590563
[2900]	valid_0's auc: 0.751181	valid_0's binary_logloss: 0.590564
[2901]	valid_0's auc: 0.751189	valid_0's binary_logloss: 0.590557
[2902]	valid_0's auc: 0.751186	valid_0's binary_logloss: 0.590562
[2903]	valid_0's auc: 0.751165	valid_0's binary_logloss: 0.590582
[2904]	valid_0's auc: 0.751157	valid_0's binary_logloss: 0.590594
[2905]	valid_0's auc: 0.751162	valid_0's binary_logloss: 0.590587
[2906]	val

[3016]	valid_0's auc: 0.751008	valid_0's binary_logloss: 0.591009
[3017]	valid_0's auc: 0.751007	valid_0's binary_logloss: 0.591012
[3018]	valid_0's auc: 0.751006	valid_0's binary_logloss: 0.591017
[3019]	valid_0's auc: 0.750994	valid_0's binary_logloss: 0.591027
[3020]	valid_0's auc: 0.750995	valid_0's binary_logloss: 0.591033
[3021]	valid_0's auc: 0.750985	valid_0's binary_logloss: 0.591044
[3022]	valid_0's auc: 0.75098	valid_0's binary_logloss: 0.591051
[3023]	valid_0's auc: 0.750974	valid_0's binary_logloss: 0.591059
[3024]	valid_0's auc: 0.750972	valid_0's binary_logloss: 0.591062
[3025]	valid_0's auc: 0.750974	valid_0's binary_logloss: 0.591065
[3026]	valid_0's auc: 0.750973	valid_0's binary_logloss: 0.591067
[3027]	valid_0's auc: 0.750966	valid_0's binary_logloss: 0.591076
[3028]	valid_0's auc: 0.750962	valid_0's binary_logloss: 0.591082
[3029]	valid_0's auc: 0.750972	valid_0's binary_logloss: 0.591071
[3030]	valid_0's auc: 0.750971	valid_0's binary_logloss: 0.591077
[3031]	vali

[3141]	valid_0's auc: 0.750777	valid_0's binary_logloss: 0.591511
[3142]	valid_0's auc: 0.750779	valid_0's binary_logloss: 0.591513
[3143]	valid_0's auc: 0.750784	valid_0's binary_logloss: 0.591514
[3144]	valid_0's auc: 0.750789	valid_0's binary_logloss: 0.591512
[3145]	valid_0's auc: 0.750793	valid_0's binary_logloss: 0.59151
[3146]	valid_0's auc: 0.750772	valid_0's binary_logloss: 0.591536
[3147]	valid_0's auc: 0.750768	valid_0's binary_logloss: 0.591548
[3148]	valid_0's auc: 0.750775	valid_0's binary_logloss: 0.591546
[3149]	valid_0's auc: 0.750765	valid_0's binary_logloss: 0.591558
[3150]	valid_0's auc: 0.750757	valid_0's binary_logloss: 0.591569
[3151]	valid_0's auc: 0.750758	valid_0's binary_logloss: 0.591569
[3152]	valid_0's auc: 0.750741	valid_0's binary_logloss: 0.591585
[3153]	valid_0's auc: 0.750749	valid_0's binary_logloss: 0.591579
[3154]	valid_0's auc: 0.750755	valid_0's binary_logloss: 0.59158
[3155]	valid_0's auc: 0.750746	valid_0's binary_logloss: 0.591592
[3156]	valid

[3267]	valid_0's auc: 0.750584	valid_0's binary_logloss: 0.592035
[3268]	valid_0's auc: 0.75058	valid_0's binary_logloss: 0.592037
[3269]	valid_0's auc: 0.75059	valid_0's binary_logloss: 0.592029
[3270]	valid_0's auc: 0.75058	valid_0's binary_logloss: 0.592043
[3271]	valid_0's auc: 0.750564	valid_0's binary_logloss: 0.592058
[3272]	valid_0's auc: 0.750545	valid_0's binary_logloss: 0.59208
[3273]	valid_0's auc: 0.750534	valid_0's binary_logloss: 0.592094
[3274]	valid_0's auc: 0.750526	valid_0's binary_logloss: 0.592109
[3275]	valid_0's auc: 0.750526	valid_0's binary_logloss: 0.592111
[3276]	valid_0's auc: 0.75053	valid_0's binary_logloss: 0.592105
[3277]	valid_0's auc: 0.75052	valid_0's binary_logloss: 0.592115
[3278]	valid_0's auc: 0.750514	valid_0's binary_logloss: 0.592122
[3279]	valid_0's auc: 0.750506	valid_0's binary_logloss: 0.592133
[3280]	valid_0's auc: 0.750502	valid_0's binary_logloss: 0.592136
[3281]	valid_0's auc: 0.750493	valid_0's binary_logloss: 0.592145
[3282]	valid_0's

KeyboardInterrupt: 

In [ ]:
pred = clf.predict_proba(X_test)[:,1]
roc_auc_score(y_test, pred)

In [ ]:
sorted_f = np.argsort(clf.feature_importances_)

for elt in range(len(sorted_f)):
    print(clf.feature_importances_[sorted_f[-elt-1]], clf.feature_name_[sorted_f[-elt-1]])

In [ ]:
import tensorflow as tf
from tabnet import *

tabnet_encoder = TabNet(
        num_features = X_test.shape[1],
        feature_dim = 256,
        output_dim = 128,
        feature_columns = None,
        n_step = 6,
        n_total = 4,
        n_shared = 2,
        relaxation_factor = 1.5,
        bn_epsilon = 1e-5,
        bn_momentum = 0.7,
        bn_virtual_divider = 10,
    )


# inputs = [tf.keras.Input(shape=(1,)) for i in range(4)] + [tf.keras.Input(shape=(327,))]

# in_size = [14000,8,100,20]
# out_size = [128,8,30,20]

# agg = [tf.squeeze(tf.keras.layers.Embedding(in_size[i], out_size[i])(inputs[i]), axis = 1) for i in range(len(in_size))] + [inputs[-1]]
# agg = tf.keras.layers.Concatenate(axis = 1)(agg)

inputs = tf.keras.Input(shape = (X_test.shape[1]))

enc, masks = tabnet_encoder(inputs)

out = tf.keras.layers.Dense(1, activation = 'sigmoid')(enc)
model = tf.keras.Model(inputs, out)

In [ ]:
model.summary()

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.02, beta_1=0.9, beta_2=0.999)

model.compile(
        loss = 'binary_crossentropy',
        optimizer = optimizer,
        metrics = ['accuracy', 'AUC'])

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ReduceLROnPlateau

early = EarlyStopping(monitor='val_loss', min_delta=0.0001, patience=9, verbose=1, 
                                                mode='auto', restore_best_weights=True)

reduce = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=4, verbose=1, 
                           mode='auto', min_delta=0.0001, cooldown=0, min_lr=0)

callbacks =[early, reduce]

epochs = 1000
batch_size = 5000

ls = X_train.shape[0]//batch_size * batch_size
lt = X_test.shape[0]//batch_size * batch_size

# model.fit([elt[:ls] for elt in X_train], y_train[:ls], 
#           validation_data = ([elt[:lt] for elt in X_test], y_test[:lt]), 
#           batch_size = batch_size, epochs = epochs, callbacks=callbacks)

model.fit(X_train.values[:ls], y_train[:ls], 
          validation_data = (X_test.values[:lt], y_test[:lt]), 
          batch_size = batch_size, epochs = epochs, callbacks=callbacks)